[Home](../index.ipynb) / BlueTooth Sniffer
***
# BlueTooth Sniffer

---
## Documents
* micropython.org: [BLE and MicroPython](https://docs.micropython.org/en/latest/library/bluetooth.html)
* bluetooth.com: Company identifiers [Local](doc/CompanyIdentfiers.csv) and [Online](https://www.bluetooth.com/specifications/assigned-numbers/company-identifiers/)
* bluetooth.com: Appearance values [Local](doc/AppearanceValues.pdf) and [Online](https://specificationrefs.bluetooth.com/assigned-values/Appearance%20Values.pdf)
* microsoft.com: Advertisement-ids [Local](doc/Advertisements.txt) and [Online](https://docs.microsoft.com/en-us/uwp/api/Windows.ApplicationModel.Background.BluetoothLEAdvertisementPublisherTrigger?view=winrt-22000)
* community.silabs.com: [Advertising: basics](https://community.silabs.com/s/article/kba-bt-0201-bluetooth-advertising-data-basics?language=en_US)
* argenox.com: [Advertising: basics](https://www.argenox.com/a-ble-advertising-primer/)

---
## BLE.gap_scan
```Python
BLE.gap_scan(duration_ms, interval_us=1280000, window_us=11250, active=False)
```
`duration_ms` 
* Run a scan operation lasting for the specified duration `duration_ms` (in milliseconds).  
* `0` : scan infinity.  
* `None` :  stop scanning


`interval_us`  
`window_us`
* The scanner will run for `window_us` microseconds every `interval_us` microseconds for a total of `duration_ms` milliseconds.  
* The default interval and window are 1.28 seconds and 11.25 milliseconds respectively (background scanning).

`active`
* can be set `True` if you want to receive scan responses in the results.


For each scan result the `_IRQ_SCAN_RESULT` event will be raised, with event data `(addr_type, addr, adv_type, rssi, adv_data)`.

`addr_type` values indicate public or random addresses:  
*   `0x00` - PUBLIC  
*   `0x01` - RANDOM (either static, RPA, or NRPA, the type is encoded in the address itself)

`addr` unique address

`adv_type` values correspond to the Bluetooth Specification:  
* `0x00` - `ADV_IND` - connectable and scannable undirected advertising  
* `0x01` - `ADV_DIRECT_IND` - connectable directed advertising  
* `0x02` - `ADV_SCAN_IND` - scannable undirected advertising  
* `0x03` - `ADV_NONCONN_IND` - non-connectable undirected advertising  
* `0x04` - `SCAN_RSP` - scan response  

`rssi` Signal strength

`adv_data` The data (description see links above)

When scanning is stopped (either due to the duration finishing or when explicitly stopped), the `_IRQ_SCAN_DONE` event will be raised.
    

---
## Source

In [ ]:
#%serialconnect --port=/dev/ttyUSB3 --baud=115200 
%serialconnect

In [ ]:
import bluetooth
from micropython import const
from machine import Pin
import binascii

# Advertising payloads are repeated packets of the following form:
#   1 byte data length (N + 1)
#   1 byte type (see constants below)
#   N bytes type-specific data

mapScan  = {}
isLocked = False


class BLESniffer:
    def __init__( self ):
        self._ADV_TYPE_FLAGS              = const(0x01)
        self._ADV_TYPE_UUID16_INCOMPLETE  = const(0x02) # Incomplete List of 16-bit Service Class UUIDs
        self._ADV_TYPE_UUID16_COMPLETE    = const(0x03) # Complete List of 16-bit Service Class UUIDs
        self._ADV_TYPE_UUID32_INCOMPLETE  = const(0x04) # Incomplete List of 32-bit Service UUIDs
        self._ADV_TYPE_UUID32_COMPLETE    = const(0x05) # Complete List of 32-bit Service UUIDs
        self._ADV_TYPE_UUID128_INCOMPLETE = const(0x06) # Incomplete List of 128-bit Services
        self._ADV_TYPE_UUID128_COMPLETE   = const(0x07) # Complete List of 128-bit Services
        

        self._ADV_TYPE_SHORT_NAME       = const(0x08) # Shortened Local Name
        self._ADV_TYPE_NAME             = const(0x09) # Complete Local Name
        self._ADV_TYPE_EXTENSION        = const(0xFF) # Manufacturer extension
        
        self._IRQ_SCAN_RESULT = const(5)
        self._IRQ_SCAN_DONE   = const(6)
        
        self._ble = bluetooth.BLE()
            
        self._ble.active(True)
        self._ble.irq(self._irq)
        
        #self.aScan = []
        
    def _irq( self, event, data ):
        global isLocked
        global mapScan
        
        if isLocked == True :
            return
           
        isLocked = True
            
        if event == self._IRQ_SCAN_RESULT:
            addr_type, addr, adv_type, rssi, adv_data = data
            # the addr, adv_data, char_data, notify_data, and uuid entries in the tuples
            # are read-only memoryview instances pointing to bluetooth’s internal ringbuffer,
            # and are only valid during the invocation of the IRQ handler

            strAdress = binascii.hexlify(bytes(addr)).decode()

            if not strAdress in mapScan :
                mapScan[strAdress] = {}
            
            # 1st byte: length of the element (excluding the length byte itself)
            # 2nd byte: AD type – specifies what data is included in the element
            # AD data – one or more bytes - the meaning is defined by AD type
            
            payload = bytes(adv_data)
            
            mapScan[strAdress][ "address_type"    ] = addr_type
            mapScan[strAdress][ "adv_type"        ] = adv_type
            mapScan[strAdress][ "signal-strength" ] = rssi
            #mapScan[strAdress][ "payload"         ] = binascii.hexlify( payload ).decode()
            
            iIndex  = 1
            advType = None
            
            while iIndex < len(payload):
                try:
                    data    = payload[iIndex+1: iIndex + payload[iIndex-1] ]
                    advType = payload[ iIndex ]
                    
                    mapScan[strAdress][ "advType" ] = advType
                    
                    if advType == self._ADV_TYPE_NAME :
                        mapScan[strAdress][ "advTypeName" ] = str(data, "utf-8")
                        
                    elif advType == self._ADV_TYPE_SHORT_NAME :
                        mapScan[strAdress][ "advTypeShortName" ] = str(data, "utf-8")
                        
                    elif advType == self._ADV_TYPE_UUID16_COMPLETE or advType == self._ADV_TYPE_UUID16_INCOMPLETE :
                        mapScan[strAdress][ "uuid16Complete" ] = bluetooth.UUID(struct.unpack("<h", data)[0])
                        
                    elif advType == self._ADV_TYPE_UUID32_COMPLETE or advType == self._ADV_TYPE_UUID32_INCOMPLETE :
                        mapScan[strAdress][ "uuid32Complete" ] = bluetooth.UUID(struct.unpack("<d", data)[0])
                        
                    elif advType == self._ADV_TYPE_UUID128_COMPLETE or advType == self._ADV_TYPE_UUID128_INCOMPLETE:
                        mapScan[strAdress][ "uuid128Complete" ] = bluetooth.UUID(data)
                        
                    elif advType == self._ADV_TYPE_EXTENSION :
                        id = data[1]<<8 | data[0]

                        mapScan[strAdress][ "manufacturer-id" ] = id
                        
                        
                        if id == 0x004C :
                            mapScan[strAdress][ "manufacturer-name"       ] = "Apple, Inc"
                            mapScan[strAdress][ "manufacturer-short-name" ] = "Apple"
                            
                        elif id == 0x00E0 :
                            mapScan[strAdress][ "manufacturer-name"       ] = "Google"
                            mapScan[strAdress][ "manufacturer-short-name" ] = "Google"
                            
                        elif id == 0x0006 :
                            mapScan[strAdress][ "manufacturer-name"       ] = "Microsoft"
                            mapScan[strAdress][ "manufacturer-short-name" ] = "MS"
                            
                        elif id == 0x0000 :
                            mapScan[strAdress][ "manufacturer-name"       ] = "Ericsson Technology Licensing"
                            mapScan[strAdress][ "manufacturer-short-name" ] = "Ericsson"
                            
                        elif id == 0x0001 :
                            mapScan[strAdress][ "manufacturer-name"       ] = "Nokia Mobile Phones"
                            mapScan[strAdress][ "manufacturer-short-name" ] = "Nokia"
                            
                        elif id == 0x0004 :
                            mapScan[strAdress][ "manufacturer-name"       ] = "Toshiba Corp."
                            mapScan[strAdress][ "manufacturer-short-name" ] = "Toshiba"
                            
                        elif id == 0x0003 :
                            mapScan[strAdress][ "manufacturer-name"       ] = "IBM Corp."
                            mapScan[strAdress][ "manufacturer-short-name" ] = "IBM"
                            
                        elif id == 0x0002 :
                            mapScan[strAdress][ "manufacturer-name"       ] = "Intel Corp."
                            mapScan[strAdress][ "manufacturer-short-name" ] = "Intel"
                            
                        elif id == 0x0075 :
                            mapScan[strAdress][ "manufacturer-name"       ] = "Samsung Electronics Co. Ltd."
                            mapScan[strAdress][ "manufacturer-short-name" ] = "Samsung"
                            
                        else:
                            if not "manufacturer-name" in mapScan[strAdress] :
                                mapScan[strAdress][ "manufacturer-name"       ] = "Unknown Manufacturer"
                                mapScan[strAdress][ "manufacturer-short-name" ] = "Unknown"

                except OSError as e:
                    aData.append( "ERROR " + e )
                iIndex += 1 + payload[iIndex-1]
                
        isLocked = False

    # Find devices
    def scan(self):
        self._ble.gap_scan(
            0,  # duration_ms: total scan duration: 0 means infinity
            30000, #interval_us = 30000, # scan every 30 mu-sec
            30000, # window_us   = 30000, # for scan-intervall: 30 mu-sec
            True   # active      = False 
        )
        
    def stop( self ):
        self._ble.gap_scan( None )
    
        
sniffer = BLESniffer()
import time

sniffer.scan()

for i in range(100):
    print( mapScan )
    time.sleep( 0.1 )
    
sniffer.stop()

print( "Done." )